In [3]:
import pyarrow.parquet as pq
from pyiceberg.catalog import load_catalog

In [10]:
! curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o ../data/yellow_tripdata_2023-01.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0  1178k      0  0:00:39  0:00:39 --:--:--  787k


In [11]:
df = pq.read_table("../data/yellow_tripdata_2023-01.parquet")

In [ ]:
print(df)

In [13]:
catalog = load_catalog(
    name="hive",
    **{
        "type": "hive",
        "uri": "thrift://hive-metastore:9083",
        "warehouse": "s3a://lakehouse",
        "s3.endpoint": "http://minio:9000",
        "s3.access-key-id": "minio",
        "s3.secret-access-key": "minio123",
        "s3.path-style-access": "true",
        "s3.region": "us-east-1",
    }
)

In [14]:
catalog.create_namespace("bronze")

In [16]:
table = catalog.create_table(
    "bronze.taxi_dataset",
    schema=df.schema,
    location="s3a://lakehouse/bronze/taxi_dataset"
)

In [17]:
table.append(df)
len(table.scan().to_arrow())

3066766